In [26]:
import numpy as np
import pandas as pd

In [27]:
# 计算各省城乡的常住人口数
L = np.array(pd.read_excel('./B0102.xls').iloc[5:, 1]).astype(float)
L = np.delete(L, 25, axis=0).reshape([30, 1]) # 删除西藏
L_ag = np.array(pd.read_excel('./B0103d.xls').iloc[5:, 1]).astype(float)
L_ag = np.delete(L_ag, 25, axis=0).reshape([30, 1]) # 删除西藏
L_na = L - L_ag

## 删除来自西藏的人口
xz_mig = [np.array(pd.read_excel('./B0701.xls').iloc[10:, 136]).astype(float)[i*4] for i in range(31)]
xz_mig_ag = [np.array(pd.read_excel('./B0701.xls').iloc[10:, 136]).astype(float)[i*4+3] for i in range(31)]
xz_mig = np.delete(xz_mig, 25, axis=0).reshape([30, 1]) # 删除西藏
xz_mig_ag = np.delete(xz_mig_ag, 25, axis=0).reshape([30, 1]) # 删除西藏
xz_mig_ag[np.isnan(xz_mig_ag)] = 0
xz_mig_na = xz_mig - xz_mig_ag
L_ag = L_ag - xz_mig_ag
L_na = L_na - xz_mig_na

In [28]:
# 构建人口流动矩阵(1)：计算省内流动
m_data = np.array(pd.read_excel('./B0701.xls').iloc[10:, 7:11]).astype(float)
m_data = np.delete(m_data, [100, 101, 102, 103], axis=0) # 删除西藏

## 城市部门
na_index = np.sort(np.concatenate([np.arange(1, 119, 4), np.arange(1, 119, 4) + 1]))
m_data_na = m_data[na_index, 0].reshape([30, 2])
m_data_na = m_data_na.sum(axis=1).reshape([30, 1])

## 农村部门
ag_index = np.arange(3, 120, 4)
m_data_ag = m_data[ag_index, 1:].sum(axis=1).reshape([30, 1])

In [29]:
# 构建人口流动矩阵(2)：计算跨省流动

m_data = np.array(pd.read_excel('./B0701.xls').iloc[10:, 11:]).astype(float)
m_data[np.isnan(m_data)] = 0
m_data = np.delete(m_data, [100, 101, 102, 103], axis=0) # 删除西藏
m_data = np.delete(m_data, [125, 126, 127, 128, 129], axis=1)
M_aux = np.ndarray([60, 150])
M = np.ndarray([60, 60])

for index in range(60):
    if index % 2 == 0:
        index_aux = index // 2
        M_aux[index, :] = m_data[4*index_aux+3, :]
    elif index % 2 == 1:
        index_aux = index // 2
        M_aux[index, :] = m_data[4*index_aux+1 : 4*index_aux+3, :].sum(axis=0)

for index in range(60):
    if index % 2 == 0:
        index_aux = index // 2
        M[:, index] = M_aux[:, 5*index_aux+1]
    elif index % 2 == 1:
        index_aux = index // 2
        M[:, index] = M_aux[:, 5*index_aux+2 : 5*index_aux+5].sum(axis=1)

In [30]:
# 构建人口流动矩阵(3)：将本地居民与省内流动数据填充到M矩阵中

## 省内流动
for index in range(60):
    if index % 2 == 0:
        index_aux = index // 2
        M[index, index+1] = m_data_ag[index_aux]
    if index % 2 == 1:
        index_aux = index // 2
        M[index, index-1] = m_data_na[index_aux]
    
## 本地居民
for index in range(60):
    if index % 2 == 0:
        index_aux = index // 2
        M[index, index] = L_ag[index_aux] - M[index, :].sum()
    if index % 2 == 1:
        index_aux = index // 2
        M[index, index] = L_na[index_aux] - M[index, :].sum()

In [31]:
# 构建人口流动矩阵(4)：构建一个DataFrame，然后导出

m = (M.T / M.sum(axis=0).reshape([60, 1])).T
m = pd.DataFrame(m)
index = [
    '北京-农业部门', '北京-非农部门', '天津-农业部门', '天津-非农部门', '河北-农业部门', '河北-非农部门',
    '山西-农业部门', '山西-非农部门', '内蒙-农业部门', '内蒙-非农部门', '辽宁-农业部门', '辽宁-非农部门',
    '吉林-农业部门', '吉林-非农部门', '黑龙江-农业部门', '黑龙江-非农部门', '上海-农业部门', '上海-非农部门',
    '江苏-农业部门', '江苏-非农部门', '浙江-农业部门', '浙江-非农部门', '安徽-农业部门', '安徽-非农部门',
    '福建-农业部门', '福建-非农部门', '江西-农业部门', '江西-非农部门', '山东-农业部门', '山东-非农部门',
    '河南-农业部门', '河南-非农部门', '湖北-农业部门', '湖北-非农部门', '湖南-农业部门', '湖南-非农部门',
    '广东-农业部门', '广东-非农部门', '广西-农业部门', '广西-非农部门', '海南-农业部门', '海南-非农部门',
    '重庆-农业部门', '重庆-非农部门', '四川-农业部门', '四川-非农部门', '贵州-农业部门', '贵州-非农部门',
    '云南-农业部门', '云南-非农部门', '陕西-农业部门', '陕西-非农部门', '甘肃-农业部门', '甘肃-非农部门',
    '青海-农业部门', '青海-非农部门', '宁夏-农业部门', '宁夏-非农部门', '新疆-农业部门', '新疆-非农部门'
]
m.index, m.columns = index, index
#m.to_excel('migration_share.xlsx')

In [34]:
# 构建人口流动矩阵(5)：计算各地各部门户籍人口，然后填充至macro_var_2010.xlsx文件中

L_bar = pd.DataFrame(M.sum(axis=0))
L_bar.columns = ['L_bar']
L_bar.index = index

In [2]:
# 使用Tombe and Zhu (2019)的流动份额数据(1)：读取原始数据，得到mij列向量各元素的oj和dk，准备后续进行匹配

raw_data = pd.read_csv('./mij2005.csv')
mij = raw_data.iloc[:, 1]

prov = [
    '安徽', '北京', '重庆', '福建', '甘肃', '广东', '广西', '贵州', '海南', '河北', '黑龙江', '河南', '湖北', '湖南', '内蒙', 
    '江苏', '江西', '吉林', '辽宁', '宁夏', '青海', '山东', '上海', '陕西', '山西', '四川', '天津', '新疆', '云南', '浙江'
]
prov_sector = []
for each_prov in prov:
    prov_sector.append(each_prov + '-农业部门')
    prov_sector.append(each_prov + '-非农部门')

dk = prov_sector * 60
oj = []
for each_prov_sector in prov_sector:
    oj = oj + [each_prov_sector]*60

mij_df = pd.concat([pd.Series(oj), pd.Series(dk), mij], axis=1)
mij_df.columns = ['oj', 'dk', 'mij']

In [3]:
# 使用Tombe and Zhu (2019)的流动份额数据(2)：构建流动份额矩阵

migration_share = pd.DataFrame(np.zeros([60, 60]))
index = [
    '北京-农业部门', '北京-非农部门', '天津-农业部门', '天津-非农部门', '河北-农业部门', '河北-非农部门',
    '山西-农业部门', '山西-非农部门', '内蒙-农业部门', '内蒙-非农部门', '辽宁-农业部门', '辽宁-非农部门',
    '吉林-农业部门', '吉林-非农部门', '黑龙江-农业部门', '黑龙江-非农部门', '上海-农业部门', '上海-非农部门',
    '江苏-农业部门', '江苏-非农部门', '浙江-农业部门', '浙江-非农部门', '安徽-农业部门', '安徽-非农部门',
    '福建-农业部门', '福建-非农部门', '江西-农业部门', '江西-非农部门', '山东-农业部门', '山东-非农部门',
    '河南-农业部门', '河南-非农部门', '湖北-农业部门', '湖北-非农部门', '湖南-农业部门', '湖南-非农部门',
    '广东-农业部门', '广东-非农部门', '广西-农业部门', '广西-非农部门', '海南-农业部门', '海南-非农部门',
    '重庆-农业部门', '重庆-非农部门', '四川-农业部门', '四川-非农部门', '贵州-农业部门', '贵州-非农部门',
    '云南-农业部门', '云南-非农部门', '陕西-农业部门', '陕西-非农部门', '甘肃-农业部门', '甘肃-非农部门',
    '青海-农业部门', '青海-非农部门', '宁夏-农业部门', '宁夏-非农部门', '新疆-农业部门', '新疆-非农部门'
]
migration_share.index, migration_share.columns = index, index

for index, row in migration_share.iterrows():
    for column in migration_share.columns:
        share = np.array(mij_df[(mij_df['oj']==index) & (mij_df['dk']==column)].mij)
        migration_share.loc[index, column] = share

migration_share.to_excel('migration_share_TZ2019.xlsx')